In [ ]:
# Gerekli kütüphaneler
import librosa
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [ ]:
def read_and_sample(csv_path, fraction):
    data = pd.read_csv(csv_path)
    sample_data = data.sample(frac=fraction, random_state=42)
    return sample_data

In [ ]:
# Verileri oku
sample_fraction = 0.5

train_data = read_and_sample("Datasets/train.csv", sample_fraction)
val_data = read_and_sample("Datasets/val.csv", sample_fraction)
test_data = read_and_sample("Datasets/test.csv", sample_fraction)

In [ ]:
# Kontrol et
print("Train Data Sample:")
print(train_data.head())
print("Validation Data Sample:")
print(val_data.head())
print("Test Data Sample:")
print(test_data.head())

In [ ]:
# Etiket dönüşüm fonksiyonu
def labels(label):
    return tf.constant([1, 0] if label == 0 else [0, 1], dtype=tf.int32)

In [ ]:
# train verileri yükle
train_x_list, train_y_list = [], []

for index, row in train_data.iterrows():
    audio, sample_rate = librosa.load(row['feature'])
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate)
    tensor_x = tf.transpose(tf.convert_to_tensor(mfccs, dtype=tf.float32))
    tensor_y = labels(row['labels'])
    train_x_list.append(tensor_x)
    train_y_list.append(tensor_y)

In [ ]:
# val verileri yükle
val_x_list, val_y_list = [], []

for index, row in val_data.iterrows():
    audio, sample_rate = librosa.load(row['feature'])
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate)
    tensor_x = tf.transpose(tf.convert_to_tensor(mfccs, dtype=tf.float32))
    tensor_y = labels(row['labels'])
    val_x_list.append(tensor_x)
    val_y_list.append(tensor_y)

In [ ]:
# Parametreleri ayarla
batch_size = 32
epochs = 100
input_size = 20
hidden_size = 256
output_size = 2
learning_rate = 1e-4

In [ ]:
# train_dataset oluştur
def generator_train():
    for features, labels in zip(train_x_list, train_y_list):
        yield (features, labels)

train_dataset = tf.data.Dataset.from_generator(
    generator_train,
    output_types=(tf.float32, tf.int16),
    output_shapes=((None, input_size), (output_size))
)

train_dataset = train_dataset.padded_batch(batch_size=batch_size, padded_shapes=([None, input_size], [output_size]))

In [ ]:
# val_dataset oluştur
def generator_val():
    for features, labels in zip(val_x_list, val_y_list):
        yield (features, labels)
        
val_dataset = tf.data.Dataset.from_generator(
    generator_val,
    output_types=(tf.float32, tf.int32),
    output_shapes=((None, input_size), (output_size))
)

val_dataset = val_dataset.padded_batch(batch_size=batch_size, padded_shapes=([None, input_size], [2]))       

In [ ]:
# Modeli oluştur
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, input_size), batch_size=None),
    tf.keras.layers.LSTM(hidden_size,tf.keras.regularizers.L2(l2=0.01)),
    tf.keras.layers.Dense(output_size, activation='linear',kernel_regularizer=tf.keras.regularizers.L2(l2=0.01))
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Model parametrelerini kaydet
checkpoint_path = "Checkpoint/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
per_epoch = 5

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=per_epoch*batch_size)

model.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
# Modeli eğit
history = model.fit(train_dataset, epochs=epochs, 
                    validation_data=val_dataset, 
                    callbacks=[cp_callback], 
                    verbose=0)

In [ ]:
# Loss grafiği çizdir
loss = history.history['loss']
val_loss = history.history['val_loss']

fig, ax = plt.subplots(1, 1)

ax.plot(loss, label='Loss')
ax.plot(val_loss, label='Validation Loss')
ax.set_title('Loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.legend()

plt.show()

In [ ]:
# Validation Accuracy grafiği çizdir
val_acc = history.history['val_accuracy']

ax.plot(val_acc, label='Validation Accuracy')
ax.set_title('Validation Accuracy')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.legend()

plt.show()

In [ ]:
# test verileri yükle
test_x_list, test_y_list = [], []

for index, row in test_data.iterrows():
    audio, sample_rate = librosa.load(row['features'])
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate)
    tensor_x = tf.transpose(tf.convert_to_tensor(mfccs, dtype=tf.float32))
    tensor_y = labels(row['labels'])
    test_x_list.append(tensor_x)
    test_y_list.append(tensor_y)

In [ ]:
# test_dataset oluştur
def generator_test():
    for features, labels in zip(test_x_list, test_y_list):
        yield (features, labels)

test_dataset = tf.data.Dataset.from_generator(
    generator_test,
    output_types=(tf.float32, tf.int16),
    output_shapes=((None, input_size), (output_size))
)

test_dataset = test_dataset.padded_batch(batch_size=batch_size, padded_shapes=([None, input_size], [output_size]))

In [ ]:
loss, acc = model.evaluate(test_dataset, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))